In [1]:
from trainingLib import *

W1009 15:09:27.479140 20296 deprecation_wrapper.py:119] From C:\Users\Zifeng Li\Desktop\HelmsleyDocker\Helmsley3DMIP\_Scripts\Utilities\networkTrain\test_model.py:4: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W1009 15:09:27.481173 20296 deprecation_wrapper.py:119] From C:\Users\Zifeng Li\Desktop\HelmsleyDocker\Helmsley3DMIP\_Scripts\Utilities\networkTrain\test_model.py:4: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.



In [2]:
# Run this block to prepare all the training data
pruneSystem()

Deleting old training files and temp files.
Deleting directory ./sampledDataset failed
Deleting directory ./sampledLabel failed


In [3]:
dicomPaths, maskPaths = getDataPaths()

In [4]:
createTrainingData(dicomPaths, maskPaths, 1, tileN=4)

Successfully created the directory ./tmp 
Successfully created the directory ./tmp/trainingImage 
Successfully created the directory ./tmp/trainingMask 
Processing data series 0
1603 701
Processing data series 1
1350 506
Processing data series 2
849 623
Processing data series 3
1212 580
Processing data series 4
372 396


In [5]:
trainUnet(COLOR_DICT, input_size=(128, 128, 3))

W1009 15:10:13.006952 20296 deprecation.py:506] From C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\tensorflow\python\ops\init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14631357706485766816
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6667611669
locality {
  bus_id: 1
  links {
  }
}
incarnation: 554379723446033007
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)     

Found 8192 images belonging to 1 classes.
Found 8192 images belonging to 1 classes.
Epoch 1/5
4096/4096 [==============================] - 825s 201ms/step - loss: 2.6326 - acc: 0.8823
Epoch 2/5
4096/4096 [==============================] - 828s 202ms/step - loss: 0.2877 - acc: 0.9655
Epoch 3/5
4096/4096 [==============================] - 819s 200ms/step - loss: 0.1552 - acc: 0.9683
Epoch 4/5
4096/4096 [==============================] - 833s 203ms/step - loss: 0.1285 - acc: 0.9695
Epoch 5/5
4096/4096 [==============================] - 821s 200ms/step - loss: 0.1200 - acc: 0.9699 5:33 - loss: 0.1202 - acc: 0. - E - ETA: 5:25 - loss: 0.1201 - acc - ETA: 5:15 - - ETA: 4:44 - loss: 0.1200 -  - ETA: 4:42 - loss: 0.1199 -  - ETA - ETA: 4:33 - loss: 0.1201 - - ETA: 4:31 - loss: 0.1202 - acc: - ETA: 4:2 - ETA: 10s - loss: 0.12 - ETA: 7s - loss: 0.1201 - acc - ETA: 5s -


In [2]:
createTestingData(1, "pa3_lava_post3", tileN=4)

Creating testing PNG files.
Successfully created the directory ./tmp/testingImage 

 Done! Converted 1792 images.


In [3]:
testUnet(COLOR_DICT, "./model_archive/segmentation_10_08_12_59.hdf5", (128, 128, 3))

W1009 12:21:46.716256 20352 deprecation.py:506] From C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\tensorflow\python\ops\init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1051910902904324287
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6667611669
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5622826111562313414
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)     

1792/1792 [==============================] - 28s 16ms/step


C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:17.531494 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Successfully created the directory ./predicted_segmnetation_10_09_12_21 


...

C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1692.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:30.590461 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1693.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:30.597487 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1694.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:30.605464 20352 uti

C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1714.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:30.798462 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1715.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:30.807460 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1716.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:30.814462 20352 uti

W1009 12:22:30.979462 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W1009 12:22:30.990461 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1738.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:30.998464 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1739.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:31.007460 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0,

W1009 12:22:31.144460 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1759.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:31.152461 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1760.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:31.159461 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: 

W1009 12:22:31.319463 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1781.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:31.327462 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_10_09_12_21\1782.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W1009 12:22:31.337462 20352 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: 

In [2]:
combineTiles("./predicted_segmnetation_10_09_12_21", 4)

(128, 128, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 

In [ ]:
()